In [132]:
import numpy as np
import pandas as pd

In [133]:
players=pd.read_csv('euroleague_players.csv')
teams=pd.read_csv('euroleague_teams.csv')
ratings=pd.read_csv('ratings.csv')

In [134]:
teams

,id,TEAM,Code,GP,Min,FGA,FTA,TR,Ast,ST,BLK,PF,Pts
0,AEK2000-2001,AEK Athens,AEK,18.0,3625.0,1082.0,473.0,593.0,154.0,121.0,23.0,452.0,1383.0
1,BSK2000-2001,Tau Ceramica Vitoria-Gasteiz,BSK,22.0,4400.0,1250.0,542.0,719.0,253.0,237.0,32.0,534.0,1704.0
2,BVP2000-2001,Buducnost,BVP,12.0,2400.0,763.0,293.0,381.0,127.0,105.0,22.0,269.0,969.0
3,CHA2000-2001,Region Wallone Spirou Charleroi,CHA,10.0,2000.0,627.0,193.0,316.0,64.0,55.0,8.0,248.0,769.0
4,COL2000-2001,Union Olimpija Ljubljana,COL,15.0,3014.0,904.0,358.0,502.0,165.0,173.0,36.0,400.0,1202.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,PAO2021-2022,Panathinaikos OPAP Athens,PAO,32.0,6400.0,1893.0,507.0,1039.0,487.0,172.0,100.0,597.0,2382.0
490,RMB2021-2022,Real Madrid,RMB,38.0,7675.0,2421.0,562.0,1455.0,686.0,241.0,127.0,744.0,2999.0
491,UNK2021-2022,UNICS Kazan,UNK,25.0,5075.0,1557.0,344.0,815.0,412.0,235.0,47.0,500.0,1917.0
492,ZAL2021-2022,Zalgiris Kaunas,ZAL,32.0,6450.0,1892.0,461.0,1011.0,532.0,178.0,59.0,653.0,2337.0


In [135]:
players['year']=players['id'].apply(lambda x: x[-9:])
players['team_id']=players['TEAM']+players['year']

In [136]:
data=players.merge(teams,left_on='team_id',right_on='id',how='left',suffixes=(None, '_TM'))

In [137]:
data.drop(columns=['id_TM','Code'],inplace=True)

In [138]:
data.rename(columns={'Ast':'AST_TM','Pts':'PTS_TM','NAME':'Player_Name','id':'Player_id','TEAM':'TEAM_CODE','TEAM_TM':'TEAM_Name'},inplace=True)

In [139]:
data=data[['Player_Name','TEAM_Name','TEAM_CODE','Player_id','team_id','year','ROLE', 'GP', 'Min', 'FGA','3PM', 'FTA', 'OR', 'DR', 'TR', 'AST', 'ST', 'BLK', 'TO', 'PF', 'PTS','GP_TM', 'Min_TM', 'FGA_TM', 'FTA_TM','TR_TM', 'AST_TM', 'ST_TM', 'BLK_TM', 'PF_TM', 'PTS_TM']]

In [140]:
ratings['id']=ratings['Code']+ratings['Year']

In [141]:
data=data.merge(ratings,left_on='team_id',right_on='id',how='left')

In [142]:
data.drop(columns=['TEAM', 'Code','Year','id'],inplace=True)

In [143]:
data['TSA']=data['FGA']+data['FTA']*0.44
# 0.44 comes from coefficients

In [144]:
data['Pt/TSA']=data['PTS']/data['TSA']
data['Pt/TSA']=data['Pt/TSA'].fillna(0)

In [145]:
data['Tm Pts/TSA']=data['PTS_TM']/(data['FGA_TM']+data['FTA_TM']*0.4)

In [146]:
data['Adj. Pts']=((data['Pt/TSA']-data['Tm Pts/TSA'])+1)*data['TSA']
data['Adj. Pts']=data['Adj. Pts'].apply(lambda x: 0 if x<0 else x)

In [147]:
data['Possessions']=data['Min']*data['Pace']/40

In [148]:
data['ThreshPts']=data['TSA']*(data['Pt/TSA']-(data['Tm Pts/TSA']-0.33))

In [149]:
data['Adj Pts_100']=data['Adj. Pts']/data['Possessions']*100
data['FGA_100']=data['FGA']/data['Possessions']*100
data['FTA_100']=data['FTA']/data['Possessions']*100
data['3PM_100']=data['3PM']/data['Possessions']*100
data['AST_100']=data['AST']/data['Possessions']*100
data['TO_100']=data['TO']/data['Possessions']*100
data['OR_100']=data['OR']/data['Possessions']*100
data['DR_100']=data['DR']/data['Possessions']*100
data['TR_100']=data['TR']/data['Possessions']*100
data['ST_100']=data['ST']/data['Possessions']*100
data['BLK_100']=data['BLK']/data['Possessions']*100
data['PF_100']=data['PF']/data['Possessions']*100

data['Adj Pts_100']=data['Adj Pts_100'].fillna(0)
data['FGA_100']=data['FGA_100'].fillna(0)
data['FTA_100']=data['FTA_100'].fillna(0)
data['3PM_100']=data['3PM_100'].fillna(0)
data['AST_100']=data['AST_100'].fillna(0)
data['TO_100']=data['TO_100'].fillna(0)
data['OR_100']=data['OR_100'].fillna(0)
data['DR_100']=data['DR_100'].fillna(0)
data['TR_100']=data['TR_100'].fillna(0)
data['ST_100']=data['ST_100'].fillna(0)
data['BLK_100']=data['BLK_100'].fillna(0)
data['PF_100']=data['PF_100'].fillna(0)

In [150]:
data['%Min']=data['Min']/(data['Min_TM']/5)
data['%TR']=data['TR']/data['TR_TM']/data['%Min']
data['%ST']=data['ST']/data['ST_TM']/data['%Min']
data['%PF']=data['PF']/data['PF_TM']/data['%Min']
data['%AST']=data['AST']/data['AST_TM']/data['%Min']
data['%BLK']=data['BLK']/data['BLK_TM']/data['%Min']

In [151]:
threshpts_tm=pd.DataFrame(data.groupby('team_id')['ThreshPts'].sum())

In [152]:
data=data.merge(threshpts_tm,left_on='team_id',right_on='team_id',how='left')

In [153]:
data.rename(columns={'ThreshPts_x':'ThreshPts','ThreshPts_y':'ThreshPts_TM'},inplace=True)

In [154]:
data['%ThreshPts']=data['ThreshPts']/data['ThreshPts_TM']/data['%Min']

In [155]:
data['ROLE'].unique()

array(['C', 'F', 'F/PF', 'G', 'PF', 'PF/C', 'PG', 'PG/F', 'PG/SG', 'SG',
       'SG/F'], dtype=object)

In [156]:
data['Position']=data['ROLE']

In [157]:
data.loc[(data['Position'] == 'PG', 'Position')] = 1
data.loc[(data['Position'] == 'SG', 'Position')] = 2
data.loc[(data['Position'] == 'G', 'Position')] = 2
data.loc[(data['Position'] == 'F', 'Position')] = 3
data.loc[(data['Position'] == 'PF', 'Position')] = 4
data.loc[(data['Position'] == 'C', 'Position')] = 5

data.loc[(data['Position'] == 'PG/SG', 'Position')] = 1.5
data.loc[(data['Position'] == 'SG/F', 'Position')] = 2.5
data.loc[(data['Position'] == 'F/PF', 'Position')] = 3.5
data.loc[(data['Position'] == 'PF/C', 'Position')] = 4.5
data.loc[(data['Position'] == 'PG/F', 'Position')] = 3

In [158]:
data['Est Pos 1']=2.13+8.668*data['%TR']-2.486*data['%ST']+0.992*data['%PF']-3.536*data['%AST']+1.667*data['%BLK']
data['Min Adj 1']=(data['Est Pos 1']*data['Min']+data['Position']*50)/(data['Min']+50)
data['Trim 1']=data['Min Adj 1'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [159]:
data['Tm Avg 1']=data['Min']*data['Trim 1']/data['Min_TM']

In [160]:
team_avg_1=pd.DataFrame(data.groupby('team_id')['Tm Avg 1'].sum())
data=data.merge(team_avg_1,left_on='team_id',right_on='team_id',how='left')

In [161]:
data.rename(columns={'Tm Avg 1_y':'Tm Avg 1'},inplace=True)
data.drop(columns='Tm Avg 1_x',inplace=True)

In [162]:
data['Adj Pos 2']=data['Min Adj 1']-(data['Tm Avg 1']-3)
data['Trim 2']=data['Adj Pos 2'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [163]:
data['Tm Avg 2']=data['Min']*data['Trim 2']/data['Min_TM']
team_avg_2=pd.DataFrame(data.groupby('team_id')['Tm Avg 2'].sum())
data=data.merge(team_avg_2,left_on='team_id',right_on='team_id',how='left')
data.rename(columns={'Tm Avg 2_y':'Tm Avg 2'},inplace=True)
data.drop(columns='Tm Avg 2_x',inplace=True)

In [164]:
data['Adj Pos 3']=data['Min Adj 1']-(data['Tm Avg 1']-3)-(data['Tm Avg 2']-3)
data['Trim 3']=data['Adj Pos 3'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [165]:
data['Tm Avg 3']=data['Min']*data['Trim 3']/data['Min_TM']
team_avg_3=pd.DataFrame(data.groupby('team_id')['Tm Avg 3'].sum())
data=data.merge(team_avg_3,left_on='team_id',right_on='team_id',how='left')
data.rename(columns={'Tm Avg 3_y':'Tm Avg 3'},inplace=True)
data.drop(columns='Tm Avg 3_x',inplace=True)

In [166]:
data['Adj Pos 4']=data['Min Adj 1']-(data['Tm Avg 1']-3)-(data['Tm Avg 2']-3)-(data['Tm Avg 3']-3)
data['Est Position']=data['Adj Pos 4'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [167]:
data['Est Off Role 1']=6-6.642*data['%AST']-8.544*data['%ThreshPts']
data['Min Adj 1 Off']=(data['Est Off Role 1']*data['Min']+4*50)/(data['Min']+50)
data['Trim 1 Off']=data['Min Adj 1 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [168]:
data['Tm Avg 1 Off']=data['Min']*data['Trim 1 Off']/data['Min_TM']
team_avg_off_1=pd.DataFrame(data.groupby('team_id')['Tm Avg 1 Off'].sum())
data=data.merge(team_avg_off_1,left_on='team_id',right_on='team_id',how='left')
data.rename(columns={'Tm Avg 1 Off_y':'Tm Avg 1 Off'},inplace=True)
data.drop(columns='Tm Avg 1 Off_x',inplace=True)

In [169]:
data['Adj Pos 2 Off']=data['Min Adj 1 Off']-(data['Tm Avg 1 Off']-3)
data['Trim 2 Off']=data['Adj Pos 2 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [170]:
data['Tm Avg 2 Off']=data['Min']*data['Trim 2 Off']/data['Min_TM']
team_avg_off_2=pd.DataFrame(data.groupby('team_id')['Tm Avg 2 Off'].sum())
data=data.merge(team_avg_off_2,left_on='team_id',right_on='team_id',how='left')
data.rename(columns={'Tm Avg 2 Off_y':'Tm Avg 2 Off'},inplace=True)
data.drop(columns='Tm Avg 2 Off_x',inplace=True)

In [171]:
data['Adj Pos 3 Off']=data['Min Adj 1 Off']-(data['Tm Avg 1 Off']-3)-(data['Tm Avg 2 Off']-3)
data['Trim 3 Off']=data['Adj Pos 3 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [172]:
data['Tm Avg 3 Off']=data['Min']*data['Trim 3 Off']/data['Min_TM']
team_avg_off_3=pd.DataFrame(data.groupby('team_id')['Tm Avg 3 Off'].sum())
data=data.merge(team_avg_off_3,left_on='team_id',right_on='team_id',how='left')
data.rename(columns={'Tm Avg 3 Off_y':'Tm Avg 3 Off'},inplace=True)
data.drop(columns='Tm Avg 3 Off_x',inplace=True)

In [173]:
data['Adj Pos 4 Off']=data['Min Adj 1 Off']-(data['Tm Avg 1 Off']-3)-(data['Tm Avg 2 Off']-3)-(data['Tm Avg 3 Off']-3)
data['Off Role']=data['Adj Pos 4 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

In [174]:
data['Adj Pts_Coeff']=(5-data['Est Position'])/4*0.86+(data['Est Position']-1)/4*0.86
data['FGA_Coeff']=(5-data['Off Role'])/4*(-0.56)+(data['Off Role']-1)/4*(-0.78)
data['FTA_Coeff']=(5-data['Off Role'])/4*(-0.246)+(data['Off Role']-1)/4*(-0.343)
data['3PM_Coeff']=(5-data['Est Position'])/4*0.389+(data['Est Position']-1)/4*0.389
data['AST_Coeff']=(5-data['Est Position'])/4*0.58+(data['Est Position']-1)/4*1.034
data['TO_Coeff']=(5-data['Est Position'])/4*(-0.964)+(data['Est Position']-1)/4*(-0.964)
data['OR_Coeff']=(5-data['Est Position'])/4*0.613+(data['Est Position']-1)/4*0.181
data['DR_Coeff']=(5-data['Est Position'])/4*0.116+(data['Est Position']-1)/4*0.181
data['ST_Coeff']=(5-data['Est Position'])/4*1.369+(data['Est Position']-1)/4*1.008
data['BLK_Coeff']=(5-data['Est Position'])/4*1.327+(data['Est Position']-1)/4*0.703
data['PF_Coeff']=(5-data['Est Position'])/4*(-0.367)+(data['Est Position']-1)/4*(-0.367)

In [175]:
data['Scoring_BPM']=data['Adj Pts_Coeff']*data['Adj Pts_100']+data['FGA_Coeff']*data['FGA_100']+data['FTA_Coeff']*data['FTA_100']+data['3PM_Coeff']*data['3PM_100']
data['Ballhandling_BPM']=data['AST_Coeff']*data['AST_100']+data['TO_Coeff']*data['TO_100']
data['Rebounding_BPM']=data['OR_Coeff']*data['OR_100']+data['DR_Coeff']*data['DR_100']
data['Defense_BPM']=data['ST_Coeff']*data['ST_100']+data['BLK_Coeff']*data['BLK_100']++data['PF_Coeff']*data['PF_100']

In [176]:
data.loc[data['Est Position'] <3, 'Pos Constraint'] = (3-data['Est Position'])/2*(-0.818)+1.387*(data['Off Role']-3)
data.loc[data['Est Position'] >=3, 'Pos Constraint'] = 1.387*(data['Off Role']-3)

In [177]:
data['Raw BPM']=data['Scoring_BPM']+data['Ballhandling_BPM']+data['Rebounding_BPM']+data['Defense_BPM']+data['Pos Constraint']
data['Contribution']=data['Raw BPM']*data['%Min']

In [178]:
data['Adj Pts_Coeff_Off']=(5-data['Est Position'])/4*0.605+(data['Est Position']-1)/4*0.605
data['FGA_Coeff_Off']=(5-data['Off Role'])/4*(-0.33)+(data['Off Role']-1)/4*(-0.472)
data['FTA_Coeff_Off']=(5-data['Off Role'])/4*(-0.145)+(data['Off Role']-1)/4*(-0.208)
data['3PM_Coeff_Off']=(5-data['Est Position'])/4*0.477+(data['Est Position']-1)/4*0.477
data['AST_Coeff_Off']=(5-data['Est Position'])/4*0.476+(data['Est Position']-1)/4*0.476
data['TO_Coeff_Off']=(5-data['Est Position'])/4*(-0.579)+(data['Est Position']-1)/4*(-0.882)
data['OR_Coeff_Off']=(5-data['Est Position'])/4*0.606+(data['Est Position']-1)/4*0.422
data['DR_Coeff_Off']=(5-data['Est Position'])/4*(-0.112)+(data['Est Position']-1)/4*0.103
data['ST_Coeff_Off']=(5-data['Est Position'])/4*0.177+(data['Est Position']-1)/4*0.294
data['BLK_Coeff_Off']=(5-data['Est Position'])/4*0.725+(data['Est Position']-1)/4*0.097
data['PF_Coeff_Off']=(5-data['Est Position'])/4*(-0.439)+(data['Est Position']-1)/4*(-0.439)

In [179]:
data['Scoring_BPM_Off']=data['Adj Pts_Coeff_Off']*data['Adj Pts_100']+data['FGA_Coeff_Off']*data['FGA_100']+data['FTA_Coeff_Off']*data['FTA_100']+data['3PM_Coeff_Off']*data['3PM_100']
data['Ballhandling_BPM_Off']=data['AST_Coeff_Off']*data['AST_100']+data['TO_Coeff_Off']*data['TO_100']
data['Rebounding_BPM_Off']=data['OR_Coeff_Off']*data['OR_100']+data['DR_Coeff_Off']*data['DR_100']
data['Defense_BPM_Off']=data['ST_Coeff_Off']*data['ST_100']+data['BLK_Coeff_Off']*data['BLK_100']++data['PF_Coeff_Off']*data['PF_100']

In [180]:
data.loc[data['Est Position'] <3, 'Pos Constraint_Off'] = (3-data['Est Position'])/2*(-1.698)+0.43*(data['Off Role']-3)
data.loc[data['Est Position'] >=3, 'Pos Constraint_Off'] = 0.43*(data['Off Role']-3)

In [181]:
data['Raw OBPM']=data['Scoring_BPM_Off']+data['Ballhandling_BPM_Off']+data['Rebounding_BPM_Off']+data['Defense_BPM_Off']+data['Pos Constraint_Off']
data['Contribution_Off']=data['Raw OBPM']*data['%Min']

In [182]:
contribution_TM=pd.DataFrame(data.groupby('team_id')['Contribution'].sum())
data=data.merge(contribution_TM,left_on='team_id',right_on='team_id',how='left')

In [183]:
data.rename(columns={'Contribution_x':'Contribution','Contribution_y':'Contribution_TM'},inplace=True)

In [184]:
data['Avg Lead']=data['Net Rat']*data['Pace']/100/2
data['Lead Bonus']=0.35/2*data['Avg Lead']
data['Adj Net Rat']=data['Net Rat']+data['Lead Bonus']
data['Tm Adj']=(data['Adj Net Rat']-data['Contribution_TM'])/5

In [185]:
data['BPM']=data['Raw BPM']+data['Tm Adj']

In [186]:
contribution_Off_TM=pd.DataFrame(data.groupby('team_id')['Contribution_Off'].sum())
data=data.merge(contribution_Off_TM,left_on='team_id',right_on='team_id',how='left')
data.rename(columns={'Contribution_Off_x':'Contribution_Off','Contribution_Off_y':'Contribution_Off_TM'},inplace=True)

In [187]:
off_rat2=pd.pivot_table(ratings,values='Off Rat',index=['Year'],aggfunc='mean')

In [188]:
data=data.merge(off_rat2,left_on='year',right_on='Year',how='left')
data.rename(columns={'Off Rat_x':'Off Rat','Off Rat_y':'Avg Off Rat'},inplace=True)

In [189]:
data['Off Rat 2']=data['Off Rat']-data['Avg Off Rat']

In [190]:
data['Adj Off Rat']=data['Off Rat 2']+data['Lead Bonus']/2
data['Tm Adj_Off']=(data['Adj Off Rat']-data['Contribution_Off_TM'])/5

In [191]:
data['OBPM']=data['Raw OBPM']+data['Tm Adj_Off']

In [192]:
data['DBPM']=data['BPM']-data['OBPM']

In [193]:
data['Contribution_final']=data['BPM']*data['%Min']

In [194]:
data['VORP']=(data['BPM']+2)*data['%Min']

In [131]:
data[['Player_Name','OBPM','DBPM','BPM','VORP']].sort_values(by='VORP',ascending=False).head(50)

,Player_Name,OBPM,DBPM,BPM,VORP
238,Dejan Tomasevic,9.725352,0.517963,10.243315,10.483338
4556,Shane Larkin,10.316141,1.176144,11.492285,8.982864
2206,Dereck Hamilton,6.022113,1.361649,7.383762,8.943536
4872,Anthony Parker,5.469504,1.620712,7.090217,7.897126
4927,Alphonso Ford,6.636984,0.399915,7.036899,7.865449
3022,Andrei Kirilenko,7.308254,4.184418,11.492672,7.760192
4778,Jaka Lakovic,7.288594,0.289037,7.577630,7.712959
6874,Louis Bullock,8.444639,0.015093,8.459733,7.505128
401,Boban Marjanovic,7.674977,1.111360,8.786337,7.276057
5011,Dimitris Diamantidis,3.935683,4.043462,7.979146,7.266066


In [273]:
def get_bpm(country):
    pl='players_'+country+'.csv'
    te='team_'+country+'.csv'
    players=pd.read_csv(pl)
    teams=pd.read_csv(te)

    data=players.merge(teams,left_on='New id',right_on='New id',how='left',suffixes=(None, '_TM'))

    data.drop(columns=['Team','Code','Year_TM','id'],inplace=True)
    data['Player_id']=data['Player']+data['Year']
    data.rename(columns={'Team_TM':'TEAM_Name','New id':'team_id','Code TBU':'TEAM_CODE','Player':'Player_Name','ORB':'OR','DRB':'DR','REB':'TR','Role':'ROLE','MIN':'Min','STL':'ST','TOV':'TO','Year':'year','MIN_TM':'Min_TM','REB_TM':'TR_TM','STL_TM':'ST_TM','DRtg':'Def Rat','ORtg':'Off Rat','eDiff':'Net Rat'},inplace=True)

    data=data[['Player_Name','TEAM_Name','TEAM_CODE','Player_id','team_id','year','ROLE', 'GP', 'Min', 'FGA','3PM', 'FTA', 'OR', 'DR', 'TR', 'AST', 'ST', 'BLK', 'TO', 'PF', 'PTS','GP_TM', 'Min_TM', 'FGA_TM', 'FTA_TM','TR_TM', 'AST_TM', 'ST_TM', 'BLK_TM', 'PF_TM', 'PTS_TM','Def Rat','Off Rat','Net Rat','Poss','Pace']]

    try:
        data['Min']=data['Min'].apply(lambda x: x.replace(',',''))
        data['Min'] = data['Min'].apply(pd.to_numeric, errors='coerce')
    except:
        pass
    
    min_tm=pd.DataFrame(data.groupby('team_id')['Min'].sum())
    data=data.merge(min_tm,left_on='team_id',right_on='team_id',how='left')
    data.rename(columns={'Min_x':'Min'},inplace=True)
    data['Min_TM']=data['Min_y']
    data.drop(columns='Min_y',inplace=True)

    data['TSA']=data['FGA']+data['FTA']*0.44
    # 0.44 comes from coefficients

    data['Pt/TSA']=data['PTS']/data['TSA']
    data['Pt/TSA']=data['Pt/TSA'].fillna(0)

    data['Tm Pts/TSA']=data['PTS_TM']/(data['FGA_TM']+data['FTA_TM']*0.4)

    data['Adj. Pts']=((data['Pt/TSA']-data['Tm Pts/TSA'])+1)*data['TSA']
    data['Adj. Pts']=data['Adj. Pts'].apply(lambda x: 0 if x<0 else x)

    data['Possessions']=data['Min']*data['Pace']/40

    data['ThreshPts']=data['TSA']*(data['Pt/TSA']-(data['Tm Pts/TSA']-0.33))

    data['Adj Pts_100']=data['Adj. Pts']/data['Possessions']*100
    data['FGA_100']=data['FGA']/data['Possessions']*100
    data['FTA_100']=data['FTA']/data['Possessions']*100
    data['3PM_100']=data['3PM']/data['Possessions']*100
    data['AST_100']=data['AST']/data['Possessions']*100
    data['TO_100']=data['TO']/data['Possessions']*100
    data['OR_100']=data['OR']/data['Possessions']*100
    data['DR_100']=data['DR']/data['Possessions']*100
    data['TR_100']=data['TR']/data['Possessions']*100
    data['ST_100']=data['ST']/data['Possessions']*100
    data['BLK_100']=data['BLK']/data['Possessions']*100
    data['PF_100']=data['PF']/data['Possessions']*100

    data['Adj Pts_100']=data['Adj Pts_100'].fillna(0)
    data['FGA_100']=data['FGA_100'].fillna(0)
    data['FTA_100']=data['FTA_100'].fillna(0)
    data['3PM_100']=data['3PM_100'].fillna(0)
    data['AST_100']=data['AST_100'].fillna(0)
    data['TO_100']=data['TO_100'].fillna(0)
    data['OR_100']=data['OR_100'].fillna(0)
    data['DR_100']=data['DR_100'].fillna(0)
    data['TR_100']=data['TR_100'].fillna(0)
    data['ST_100']=data['ST_100'].fillna(0)
    data['BLK_100']=data['BLK_100'].fillna(0)
    data['PF_100']=data['PF_100'].fillna(0)



    data['%Min']=data['Min']/(data['Min_TM']/5)
    data['%TR']=data['TR']/data['TR_TM']/data['%Min']
    data['%ST']=data['ST']/data['ST_TM']/data['%Min']
    data['%PF']=data['PF']/data['PF_TM']/data['%Min']
    data['%AST']=data['AST']/data['AST_TM']/data['%Min']
    data['%BLK']=data['BLK']/data['BLK_TM']/data['%Min']

    threshpts_tm=pd.DataFrame(data.groupby('team_id')['ThreshPts'].sum())

    data=data.merge(threshpts_tm,left_on='team_id',right_on='team_id',how='left')

    data.rename(columns={'ThreshPts_x':'ThreshPts','ThreshPts_y':'ThreshPts_TM'},inplace=True)

    data['%ThreshPts']=data['ThreshPts']/data['ThreshPts_TM']/data['%Min']

    data['ROLE'].unique()

    data['Position']=data['ROLE']

    data.loc[(data['Position'] == 'PG', 'Position')] = 1
    data.loc[(data['Position'] == 'SG', 'Position')] = 2
    data.loc[(data['Position'] == 'G', 'Position')] = 2
    data.loc[(data['Position'] == 'SF', 'Position')] = 3
    data.loc[(data['Position'] == 'PF', 'Position')] = 4
    data.loc[(data['Position'] == 'C', 'Position')] = 5


    data['Est Pos 1']=2.13+8.668*data['%TR']-2.486*data['%ST']+0.992*data['%PF']-3.536*data['%AST']+1.667*data['%BLK']
    data['Min Adj 1']=(data['Est Pos 1']*data['Min']+data['Position']*50)/(data['Min']+50)
    data['Trim 1']=data['Min Adj 1'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

    data['Tm Avg 1']=data['Min']*data['Trim 1']/data['Min_TM']

    team_avg_1=pd.DataFrame(data.groupby('team_id')['Tm Avg 1'].sum())
    data=data.merge(team_avg_1,left_on='team_id',right_on='team_id',how='left')

    data.rename(columns={'Tm Avg 1_y':'Tm Avg 1'},inplace=True)
    data.drop(columns='Tm Avg 1_x',inplace=True)

    data['Adj Pos 2']=data['Min Adj 1']-(data['Tm Avg 1']-3)
    data['Trim 2']=data['Adj Pos 2'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

    data['Tm Avg 2']=data['Min']*data['Trim 2']/data['Min_TM']
    team_avg_2=pd.DataFrame(data.groupby('team_id')['Tm Avg 2'].sum())
    data=data.merge(team_avg_2,left_on='team_id',right_on='team_id',how='left')
    data.rename(columns={'Tm Avg 2_y':'Tm Avg 2'},inplace=True)
    data.drop(columns='Tm Avg 2_x',inplace=True)

    data['Adj Pos 3']=data['Min Adj 1']-(data['Tm Avg 1']-3)-(data['Tm Avg 2']-3)
    data['Trim 3']=data['Adj Pos 3'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

    data['Tm Avg 3']=data['Min']*data['Trim 3']/data['Min_TM']
    team_avg_3=pd.DataFrame(data.groupby('team_id')['Tm Avg 3'].sum())
    data=data.merge(team_avg_3,left_on='team_id',right_on='team_id',how='left')
    data.rename(columns={'Tm Avg 3_y':'Tm Avg 3'},inplace=True)
    data.drop(columns='Tm Avg 3_x',inplace=True)

    data['Adj Pos 4']=data['Min Adj 1']-(data['Tm Avg 1']-3)-(data['Tm Avg 2']-3)-(data['Tm Avg 3']-3)
    data['Est Position']=data['Adj Pos 4'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

    data['Est Off Role 1']=6-6.642*data['%AST']-8.544*data['%ThreshPts']
    data['Min Adj 1 Off']=(data['Est Off Role 1']*data['Min']+4*50)/(data['Min']+50)
    data['Trim 1 Off']=data['Min Adj 1 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

    data['Tm Avg 1 Off']=data['Min']*data['Trim 1 Off']/data['Min_TM']
    team_avg_off_1=pd.DataFrame(data.groupby('team_id')['Tm Avg 1 Off'].sum())
    data=data.merge(team_avg_off_1,left_on='team_id',right_on='team_id',how='left')
    data.rename(columns={'Tm Avg 1 Off_y':'Tm Avg 1 Off'},inplace=True)
    data.drop(columns='Tm Avg 1 Off_x',inplace=True)

    data['Adj Pos 2 Off']=data['Min Adj 1 Off']-(data['Tm Avg 1 Off']-3)
    data['Trim 2 Off']=data['Adj Pos 2 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

    data['Tm Avg 2 Off']=data['Min']*data['Trim 2 Off']/data['Min_TM']
    team_avg_off_2=pd.DataFrame(data.groupby('team_id')['Tm Avg 2 Off'].sum())
    data=data.merge(team_avg_off_2,left_on='team_id',right_on='team_id',how='left')
    data.rename(columns={'Tm Avg 2 Off_y':'Tm Avg 2 Off'},inplace=True)
    data.drop(columns='Tm Avg 2 Off_x',inplace=True)

    data['Adj Pos 3 Off']=data['Min Adj 1 Off']-(data['Tm Avg 1 Off']-3)-(data['Tm Avg 2 Off']-3)
    data['Trim 3 Off']=data['Adj Pos 3 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))

    data['Tm Avg 3 Off']=data['Min']*data['Trim 3 Off']/data['Min_TM']
    team_avg_off_3=pd.DataFrame(data.groupby('team_id')['Tm Avg 3 Off'].sum())
    data=data.merge(team_avg_off_3,left_on='team_id',right_on='team_id',how='left')
    data.rename(columns={'Tm Avg 3 Off_y':'Tm Avg 3 Off'},inplace=True)
    data.drop(columns='Tm Avg 3 Off_x',inplace=True)

    data['Adj Pos 4 Off']=data['Min Adj 1 Off']-(data['Tm Avg 1 Off']-3)-(data['Tm Avg 2 Off']-3)-(data['Tm Avg 3 Off']-3)
    data['Off Role']=data['Adj Pos 4 Off'].apply(lambda x: min(x,5)).apply(lambda x: max(x,1))


    data['Adj Pts_Coeff']=(5-data['Est Position'])/4*0.86+(data['Est Position']-1)/4*0.86
    data['FGA_Coeff']=(5-data['Off Role'])/4*(-0.56)+(data['Off Role']-1)/4*(-0.78)
    data['FTA_Coeff']=(5-data['Off Role'])/4*(-0.246)+(data['Off Role']-1)/4*(-0.343)
    data['3PM_Coeff']=(5-data['Est Position'])/4*0.389+(data['Est Position']-1)/4*0.389
    data['AST_Coeff']=(5-data['Est Position'])/4*0.58+(data['Est Position']-1)/4*1.034
    data['TO_Coeff']=(5-data['Est Position'])/4*(-0.964)+(data['Est Position']-1)/4*(-0.964)
    data['OR_Coeff']=(5-data['Est Position'])/4*0.613+(data['Est Position']-1)/4*0.181
    data['DR_Coeff']=(5-data['Est Position'])/4*0.116+(data['Est Position']-1)/4*0.181
    data['ST_Coeff']=(5-data['Est Position'])/4*1.369+(data['Est Position']-1)/4*1.008
    data['BLK_Coeff']=(5-data['Est Position'])/4*1.327+(data['Est Position']-1)/4*0.703
    data['PF_Coeff']=(5-data['Est Position'])/4*(-0.367)+(data['Est Position']-1)/4*(-0.367)

    data['Scoring_BPM']=data['Adj Pts_Coeff']*data['Adj Pts_100']+data['FGA_Coeff']*data['FGA_100']+data['FTA_Coeff']*data['FTA_100']+data['3PM_Coeff']*data['3PM_100']
    data['Ballhandling_BPM']=data['AST_Coeff']*data['AST_100']+data['TO_Coeff']*data['TO_100']
    data['Rebounding_BPM']=data['OR_Coeff']*data['OR_100']+data['DR_Coeff']*data['DR_100']
    data['Defense_BPM']=data['ST_Coeff']*data['ST_100']+data['BLK_Coeff']*data['BLK_100']++data['PF_Coeff']*data['PF_100']

    data.loc[data['Est Position'] <3, 'Pos Constraint'] = (3-data['Est Position'])/2*(-0.818)+1.387*(data['Off Role']-3)
    data.loc[data['Est Position'] >=3, 'Pos Constraint'] = 1.387*(data['Off Role']-3)

    data['Raw BPM']=data['Scoring_BPM']+data['Ballhandling_BPM']+data['Rebounding_BPM']+data['Defense_BPM']+data['Pos Constraint']
    data['Contribution']=data['Raw BPM']*data['%Min']

    data['Adj Pts_Coeff_Off']=(5-data['Est Position'])/4*0.605+(data['Est Position']-1)/4*0.605
    data['FGA_Coeff_Off']=(5-data['Off Role'])/4*(-0.33)+(data['Off Role']-1)/4*(-0.472)
    data['FTA_Coeff_Off']=(5-data['Off Role'])/4*(-0.145)+(data['Off Role']-1)/4*(-0.208)
    data['3PM_Coeff_Off']=(5-data['Est Position'])/4*0.477+(data['Est Position']-1)/4*0.477
    data['AST_Coeff_Off']=(5-data['Est Position'])/4*0.476+(data['Est Position']-1)/4*0.476
    data['TO_Coeff_Off']=(5-data['Est Position'])/4*(-0.579)+(data['Est Position']-1)/4*(-0.882)
    data['OR_Coeff_Off']=(5-data['Est Position'])/4*0.606+(data['Est Position']-1)/4*0.422
    data['DR_Coeff_Off']=(5-data['Est Position'])/4*(-0.112)+(data['Est Position']-1)/4*0.103
    data['ST_Coeff_Off']=(5-data['Est Position'])/4*0.177+(data['Est Position']-1)/4*0.294
    data['BLK_Coeff_Off']=(5-data['Est Position'])/4*0.725+(data['Est Position']-1)/4*0.097
    data['PF_Coeff_Off']=(5-data['Est Position'])/4*(-0.439)+(data['Est Position']-1)/4*(-0.439)

    data['Scoring_BPM_Off']=data['Adj Pts_Coeff_Off']*data['Adj Pts_100']+data['FGA_Coeff_Off']*data['FGA_100']+data['FTA_Coeff_Off']*data['FTA_100']+data['3PM_Coeff_Off']*data['3PM_100']
    data['Ballhandling_BPM_Off']=data['AST_Coeff_Off']*data['AST_100']+data['TO_Coeff_Off']*data['TO_100']
    data['Rebounding_BPM_Off']=data['OR_Coeff_Off']*data['OR_100']+data['DR_Coeff_Off']*data['DR_100']
    data['Defense_BPM_Off']=data['ST_Coeff_Off']*data['ST_100']+data['BLK_Coeff_Off']*data['BLK_100']++data['PF_Coeff_Off']*data['PF_100']

    data.loc[data['Est Position'] <3, 'Pos Constraint_Off'] = (3-data['Est Position'])/2*(-1.698)+0.43*(data['Off Role']-3)
    data.loc[data['Est Position'] >=3, 'Pos Constraint_Off'] = 0.43*(data['Off Role']-3)

    data['Raw OBPM']=data['Scoring_BPM_Off']+data['Ballhandling_BPM_Off']+data['Rebounding_BPM_Off']+data['Defense_BPM_Off']+data['Pos Constraint_Off']
    data['Contribution_Off']=data['Raw OBPM']*data['%Min']


    contribution_TM=pd.DataFrame(data.groupby('team_id')['Contribution'].sum())
    data=data.merge(contribution_TM,left_on='team_id',right_on='team_id',how='left')

    data.rename(columns={'Contribution_x':'Contribution','Contribution_y':'Contribution_TM'},inplace=True)

    data['Avg Lead']=data['Net Rat']*data['Pace']/100/2
    data['Lead Bonus']=0.35/2*data['Avg Lead']
    data['Adj Net Rat']=data['Net Rat']+data['Lead Bonus']
    data['Tm Adj']=(data['Adj Net Rat']-data['Contribution_TM'])/5

    data['BPM']=data['Raw BPM']+data['Tm Adj']

    contribution_Off_TM=pd.DataFrame(data.groupby('team_id')['Contribution_Off'].sum())
    data=data.merge(contribution_Off_TM,left_on='team_id',right_on='team_id',how='left')
    data.rename(columns={'Contribution_Off_x':'Contribution_Off','Contribution_Off_y':'Contribution_Off_TM'},inplace=True)

    off_rat2=pd.pivot_table(data,values='Off Rat',index=['year'],aggfunc='mean')

    data=data.merge(off_rat2,left_on='year',right_on='year',how='left')
    data.rename(columns={'Off Rat_x':'Off Rat','Off Rat_y':'Avg Off Rat'},inplace=True)

    data['Off Rat 2']=data['Off Rat']-data['Avg Off Rat']

    data['Adj Off Rat']=data['Off Rat 2']+data['Lead Bonus']/2
    data['Tm Adj_Off']=(data['Adj Off Rat']-data['Contribution_Off_TM'])/5

    data['OBPM']=data['Raw OBPM']+data['Tm Adj_Off']

    data['DBPM']=data['BPM']-data['OBPM']

    data['Contribution_final']=data['BPM']*data['%Min']

    data['VORP']=(data['BPM']+2)*data['%Min']
    
    return data

In [277]:
esp=get_bpm('esp')
esp.to_csv('players_esp_bpm.csv',index=False)

it=get_bpm('it')
it.to_csv('players_it_bpm.csv',index=False)

tur=get_bpm('tur')
tur.to_csv('players_tur_bpm.csv',index=False)

gr=get_bpm('gr')
gr.to_csv('players_gr_bpm.csv',index=False)

fr=get_bpm('fr')
fr.to_csv('players_fr_bpm.csv',index=False)

adr=get_bpm('adr')
adr.to_csv('players_adr_bpm.csv',index=False)

rus=get_bpm('rus')
rus.to_csv('players_rus_bpm.csv',index=False)